In [1]:
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

from leafy.loader import ImageLoader
from leafy.trainloader import ImageDataset

data_folder = Path("./images")
loader = ImageLoader(data_folder=data_folder)
image_db = ImageDataset(loader)
train_set, test_set = random_split(image_db, [0.8, 0.2])

# class_distribution = loader.get_better_class_distribution()
im, y = image_db[0]
num_classes = len(y)
print(f"{im.shape = }\n{y.shape = }")

Initiated loader on folder /home/joep/Code/Leafliction/images. Found 7233 images.
im.shape = torch.Size([3, 256, 256])
y.shape = torch.Size([8])


In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet50, ResNet50_Weights
from icecream import ic
from torch.nn import Module

class BasicClassifier(Module):
    def __init__(self, num_classes):
        super(BasicClassifier, self).__init__()
        resnet = resnet50(weights=ResNet50_Weights.DEFAULT)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.fc = nn.Linear(2048, 8)
        
    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x.view(x.size(0), -1))
        return x
        # return F.softmax(x, dim=1) # will break for unbatched data


net = BasicClassifier(num_classes = 8)
net = net.cuda()


preprocess = ResNet50_Weights.DEFAULT.transforms()
# resnet = resnet50(weights=ResNet50_Weights.DEFAULT)


In [4]:
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_reserved()/1024**2)

memory_stats()

89.974609375
118.0


In [5]:
from tqdm.notebook import tqdm

from torch.utils.data import DataLoader, random_split
from torch.optim import Adam

train_loader = DataLoader(train_set, 32, shuffle = True)

ce_loss = nn.CrossEntropyLoss()

optim = Adam(net.parameters())

for epoch in range(1):
    for x, y in tqdm(train_loader):
        optim.zero_grad()
        x = x.cuda()
        x = preprocess(x)
        y = y.cuda()
        y_hat = net(x)
        loss = ce_loss(y_hat, y)
        loss.backward()
        optim.step()
        print(loss)

optim.zero_grad()

  0%|          | 0/181 [00:00<?, ?it/s]

/home/joep/Code/Leafliction/venv/lib/python3.8/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor(2.0766, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.8176, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.3559, device='cuda:0', grad_fn=<DivBackward1>)
tensor(1.2820, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.7468, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.8443, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.8587, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.4458, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.5751, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.5814, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.1551, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.3776, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.4369, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.4134, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.2163, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.4991, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.1849, device='cuda:0', grad_fn=<DivBackward1>)
tensor(0.6287, device='cuda:0', grad_fn=<DivBack

In [6]:
# torch.cuda.empty_cache()
# net = net.eval()
test_loader = DataLoader(test_set, 32, shuffle = True)

gts = []
preds = []

for x, y in tqdm(test_loader):
    gts.append(torch.argmax(y, dim=1))
    y_hat = net(preprocess(x.cuda()))
    preds.append(torch.argmax(y_hat, dim=1))

# y_hat = F.softmax(net(preprocess(x)))
# print(y, y_hat)

  0%|          | 0/46 [00:00<?, ?it/s]

In [15]:
getes  = torch.concat(gts)
predes = torch.concat(preds).cpu()


In [17]:
(getes == predes).to(torch.float).mean()

tensor(0.9620)

tensor([0, 0, 4,  ..., 6, 7, 0], device='cuda:0')